In [1]:
import numpy as np
import pandas as pd
from category_encoders.cat_boost import CatBoostEncoder

In [2]:
train = pd.read_csv("../data/tabular_playground/train.csv")
test = pd.read_csv("../data/tabular_playground/test.csv")

data = pd.concat([train, test], sort=False)

In [3]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1.0,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0.0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0.0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0.0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1.0,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,199995,NaN,3,"Cash, Cheryle",female,27.00,0,0,7686,10.12,NaN,Q
99996,199996,NaN,1,"Brown, Howard",male,59.00,1,0,13004,68.31,NaN,S
99997,199997,NaN,3,"Lightfoot, Cameron",male,47.00,0,0,4383317,10.87,NaN,S
99998,199998,NaN,1,"Jacobsen, Margaret",female,49.00,1,2,PC 26988,29.68,B20828,C


In [4]:
data["Sex"].replace(["male", "female"], [0,1], inplace=True)
data["Embarked"].fillna(("S"), inplace=True)
data["Embarked"] = data["Embarked"].map(
    {"S":0, "C":1, "Q":2}
).astype(int)
data["Fare"].fillna(np.mean(
    data["Fare"]
), inplace=True)

In [5]:
data["Age"].fillna(data["Age"].median(), inplace=True)

In [6]:
data["FamilySize"] = data["Parch"] + data["SibSp"] + 1
data["IsAlone"] = 0
data.loc[data["FamilySize"] == 1, "IsAlone"] = 1

In [7]:
def convertTextToStrAndNumeric(feature):
    """
    与えられた特徴を文字列と数字部分に分割
    """
    string = ""
    numeric = ""
    feature = str(feature).replace(".", "").replace("/", "").replace(" ", "")
    for char in feature:
        if char.isnumeric():
            numeric += char
        else:
            string += char
    if numeric != "":
        return string, int(numeric)
    return string, np.nan

data["FirstName"] = data.Name.map(
    lambda x: str(x.split(",")[0])
)
data["SurName"] = data.Name.map(
    lambda x: str(x.split(",")[1])
)

for col in ['Name', 'FirstName', 'SurName']:
    data["Counter_"+col] = data[col].map(
        data.groupby(col)["PassengerId"].count().to_dict()
    )

ce = CatBoostEncoder()

data["Ticket_type"] = data["Ticket"].map(lambda x: convertTextToStrAndNumeric(x)[0])
data["Ticket_number"] = data["Ticket"].map(lambda x: convertTextToStrAndNumeric(x)[1])

data["Cabin_type"] = data["Cabin"].map(lambda x: convertTextToStrAndNumeric(x)[0])
data["Cabin_number"] = data["Cabin"].map(lambda x: convertTextToStrAndNumeric(x)[1])

train = data[:len(train)]
test = data[len(train):]
train["Ticket_type"] = ce.fit_transform(train["Ticket_type"], train["Survived"])
test["Ticket_type"] = ce.transform(test["Ticket_type"])

train["Cabin_type"] = ce.fit_transform(train["Cabin_type"], train["Survived"])
test["Cabin_type"] = ce.transform(test["Cabin_type"])

train["FirstName"] = ce.fit_transform(train["FirstName"], train["Survived"])
test["FirstName"] = ce.transform(test["FirstName"])

train["SurName"] = ce.fit_transform(train["SurName"], train["Survived"])
test["SurName"] = ce.transform(test["SurName"])

data = pd.concat([train, test], sort=False)


/tmp/ipykernel_16202/101077945.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Ticket_type"] = ce.fit_transform(train["Ticket_type"], train["Survived"])
/tmp/ipykernel_16202/101077945.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Ticket_type"] = ce.transform(test["Ticket_type"])
/tmp/ipykernel_16202/101077945.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [8]:
print(data["Ticket"])

0           209245
1            27323
2        CA 457703
3         A. 10866
4           427635
           ...    
99995         7686
99996        13004
99997      4383317
99998     PC 26988
99999     PC 41824
Name: Ticket, Length: 200000, dtype: object


In [9]:
delete_columns = ['Name', 'PassengerId', 'Cabin', 'Ticket']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,NaN,3,0,19.0,0,0,63.01,0,1,1,0.642774,0.235321,1,18,1384,0.436894,24745.0,0.342102,NaN
1,NaN,3,1,53.0,0,0,5.81,0,1,1,0.480504,0.748968,1,108,125,0.436894,13264.0,0.342102,NaN
2,NaN,1,1,19.0,0,0,38.91,1,1,1,0.456166,0.695606,1,511,244,0.436894,25990.0,0.630165,15315.0
3,NaN,2,0,25.0,0,0,12.93,0,1,1,0.394060,0.181203,1,51,770,0.436894,314011.0,0.342102,NaN
4,NaN,1,1,17.0,0,2,26.89,1,3,0,0.142580,0.674672,1,8,387,0.436894,26203.0,0.630165,22515.0


In [10]:
y_train = train["Survived"]
X_train = train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [11]:
y_train.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: Survived, dtype: float64

In [12]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,1,0,31.00,2,0,27.14,0,3,0,0.42774,0.42774,1,47,51,0.427740,209245.0,0.427740,12239.0
1,3,0,31.00,0,0,13.35,0,1,1,0.42774,0.42774,1,54,31,0.713870,27323.0,0.427740,NaN
2,3,0,0.33,1,2,71.29,0,4,0,0.42774,0.42774,2,165,1128,0.427740,457703.0,0.213870,NaN
3,3,0,19.00,0,0,13.04,0,1,1,0.42774,0.42774,2,53,4736,0.427740,10866.0,0.142580,NaN
4,3,0,25.00,0,0,7.76,0,1,1,0.42774,0.42774,1,39,3662,0.475913,427635.0,0.106935,NaN


In [13]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,3,0,19.0,0,0,63.01,0,1,1,0.642774,0.235321,1,18,1384,0.436894,24745.0,0.342102,NaN
1,3,1,53.0,0,0,5.81,0,1,1,0.480504,0.748968,1,108,125,0.436894,13264.0,0.342102,NaN
2,1,1,19.0,0,0,38.91,1,1,1,0.456166,0.695606,1,511,244,0.436894,25990.0,0.630165,15315.0
3,2,0,25.0,0,0,12.93,0,1,1,0.394060,0.181203,1,51,770,0.436894,314011.0,0.342102,NaN
4,1,1,17.0,0,2,26.89,1,3,0,0.142580,0.674672,1,8,387,0.436894,26203.0,0.630165,22515.0


In [14]:
# lightLGM + skfold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [15]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train), ))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [16]:
# categorical_features = ["Embarked", "Pclass", "Sex"]
categorical_features = ["Embarked", "Pclass", "Sex", "Ticket_type", "Cabin_type", "FirstName", "SurName", "Ticket_number", "Cabin_number"]


In [17]:
params = {
    "objective":"binary",
    "max_bin":500,
    "learning_rate":0.05,
    "num_leaves":80
}

In [18]:
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index,:]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(
        X_tr, y_tr, categorical_feature=categorical_features
    )

    lgb_eval = lgb.Dataset(
        X_val, y_val, reference=lgb_train, categorical_feature=categorical_features 
    )

    model = lgb.train(
        params, lgb_train, valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10, num_boost_round=1000, early_stopping_rounds=100
    )

    oof_train[valid_index] = model.predict(
        X_val, num_iteration=model.best_iteration
    )

    y_pred = model.predict(
        X_test, num_iteration=model.best_iteration
    )

    y_preds.append(y_pred)
    models.append(model)

/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 34219, number of negative: 45781
[LightGBM] [Info] Total Bins 8410
[LightGBM] [Info] Number of data: 80000, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.427737 -> initscore=-0.291088
[LightGBM] [Info] Start training from score -0.291088
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.566066	valid_1's binary_logloss: 0.568297
[20]	training's binary_logloss: 0.517266	valid_1's binary_logloss: 0.521276
[30]	training's binary_logloss:

In [19]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.77159

In [20]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0])

In [21]:
sub = pd.read_csv('../data/tabular_playground/sample_submission.csv')

In [22]:
sub["Survived"] = y_sub
sub.to_csv("./subs/lightgbm_skfold_isAlone_categorized.csv",index=False)

In [23]:
sub.head()

,PassengerId,Survived
0,100000,0
1,100001,1
2,100002,1
3,100003,0
4,100004,1


# Pseudolabellingを使ってみる
- 参考
    - https://www.kaggle.com/c/tabular-playground-series-apr-2021/discussion/231738

In [24]:
test.loc[:,"Survived"] = y_sub

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [25]:
test

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,0,3,0,19.0,0,0,63.01,0,1,1,0.642774,0.235321,1,18,1384,0.436894,24745.0,0.342102,NaN
1,1,3,1,53.0,0,0,5.81,0,1,1,0.480504,0.748968,1,108,125,0.436894,13264.0,0.342102,NaN
2,1,1,1,19.0,0,0,38.91,1,1,1,0.456166,0.695606,1,511,244,0.436894,25990.0,0.630165,15315.0
3,0,2,0,25.0,0,0,12.93,0,1,1,0.394060,0.181203,1,51,770,0.436894,314011.0,0.342102,NaN
4,1,1,1,17.0,0,2,26.89,1,3,0,0.142580,0.674672,1,8,387,0.436894,26203.0,0.630165,22515.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,3,1,27.0,0,0,10.12,2,1,1,0.442774,0.475913,1,34,4,0.436894,7686.0,0.342102,NaN
99996,0,1,0,59.0,1,0,68.31,0,2,0,0.433075,0.239656,2,1352,316,0.436894,13004.0,0.342102,NaN
99997,0,3,0,47.0,0,0,10.87,0,1,1,0.346820,0.409902,1,8,52,0.436894,4383317.0,0.342102,NaN
99998,1,1,1,49.0,1,2,29.68,1,4,0,0.571290,0.668847,1,12,575,0.659571,26988.0,0.630165,20828.0


In [26]:
pseudoLabeled_train = pd.concat([train, test[:int(len(test) / 2)]], sort=False)
pseudoLabeled_train = pseudoLabeled_train.reset_index(drop=True)
pseudoLabeled_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,1.0,1,0,31.00,2,0,27.14,0,3,0,0.427740,0.427740,1,47,51,0.427740,209245.0,0.427740,12239.0
1,0.0,3,0,31.00,0,0,13.35,0,1,1,0.427740,0.427740,1,54,31,0.713870,27323.0,0.427740,NaN
2,0.0,3,0,0.33,1,2,71.29,0,4,0,0.427740,0.427740,2,165,1128,0.427740,457703.0,0.213870,NaN
3,0.0,3,0,19.00,0,0,13.04,0,1,1,0.427740,0.427740,2,53,4736,0.427740,10866.0,0.142580,NaN
4,1.0,3,0,25.00,0,0,7.76,0,1,1,0.427740,0.427740,1,39,3662,0.475913,427635.0,0.106935,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0.0,3,0,54.00,0,0,10.89,0,1,1,0.458964,0.239656,2,558,316,0.542839,46893.0,0.342102,NaN
149996,0.0,3,0,19.00,0,0,10.21,0,1,1,0.340595,0.288030,1,21,283,0.436894,14451.0,0.342102,NaN
149997,1.0,3,1,4.00,1,1,36.06,0,3,0,0.475913,0.685448,1,3,392,0.436894,13284.0,0.342102,NaN
149998,1.0,1,0,30.00,0,0,65.23,1,1,1,0.475913,0.224698,1,9,1096,0.436894,25408.0,0.342102,NaN


In [27]:
y_train = pseudoLabeled_train["Survived"]
X_train = pseudoLabeled_train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [28]:
y_train

0         1.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
149995    0.0
149996    0.0
149997    1.0
149998    1.0
149999    0.0
Name: Survived, Length: 150000, dtype: float64

In [29]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,3,0,19.0,0,0,63.01,0,1,1,0.642774,0.235321,1,18,1384,0.436894,24745.0,0.342102,NaN
1,3,1,53.0,0,0,5.81,0,1,1,0.480504,0.748968,1,108,125,0.436894,13264.0,0.342102,NaN
2,1,1,19.0,0,0,38.91,1,1,1,0.456166,0.695606,1,511,244,0.436894,25990.0,0.630165,15315.0
3,2,0,25.0,0,0,12.93,0,1,1,0.394060,0.181203,1,51,770,0.436894,314011.0,0.342102,NaN
4,1,1,17.0,0,2,26.89,1,3,0,0.142580,0.674672,1,8,387,0.436894,26203.0,0.630165,22515.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,1,27.0,0,0,10.12,2,1,1,0.442774,0.475913,1,34,4,0.436894,7686.0,0.342102,NaN
99996,1,0,59.0,1,0,68.31,0,2,0,0.433075,0.239656,2,1352,316,0.436894,13004.0,0.342102,NaN
99997,3,0,47.0,0,0,10.87,0,1,1,0.346820,0.409902,1,8,52,0.436894,4383317.0,0.342102,NaN
99998,1,1,49.0,1,2,29.68,1,4,0,0.571290,0.668847,1,12,575,0.659571,26988.0,0.630165,20828.0


In [30]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName,Ticket_type,Ticket_number,Cabin_type,Cabin_number
0,1,0,31.00,2,0,27.14,0,3,0,0.427740,0.427740,1,47,51,0.427740,209245.0,0.427740,12239.0
1,3,0,31.00,0,0,13.35,0,1,1,0.427740,0.427740,1,54,31,0.713870,27323.0,0.427740,NaN
2,3,0,0.33,1,2,71.29,0,4,0,0.427740,0.427740,2,165,1128,0.427740,457703.0,0.213870,NaN
3,3,0,19.00,0,0,13.04,0,1,1,0.427740,0.427740,2,53,4736,0.427740,10866.0,0.142580,NaN
4,3,0,25.00,0,0,7.76,0,1,1,0.427740,0.427740,1,39,3662,0.475913,427635.0,0.106935,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,3,0,54.00,0,0,10.89,0,1,1,0.458964,0.239656,2,558,316,0.542839,46893.0,0.342102,NaN
149996,3,0,19.00,0,0,10.21,0,1,1,0.340595,0.288030,1,21,283,0.436894,14451.0,0.342102,NaN
149997,3,1,4.00,1,1,36.06,0,3,0,0.475913,0.685448,1,3,392,0.436894,13284.0,0.342102,NaN
149998,1,0,30.00,0,0,65.23,1,1,1,0.475913,0.224698,1,9,1096,0.436894,25408.0,0.342102,NaN


In [31]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train), ))
cv2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [32]:
for fold_id, (train_index, valid_index) in enumerate(cv2.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index,:]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(
        X_tr, y_tr, categorical_feature=categorical_features
    )

    lgb_eval = lgb.Dataset(
        X_val, y_val, reference=lgb_train, categorical_feature=categorical_features 
    )

    model = lgb.train(
        params, lgb_train, valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10, num_boost_round=1000, early_stopping_rounds=100
    )

    oof_train[valid_index] = model.predict(
        X_val, num_iteration=model.best_iteration
    )

    y_pred = model.predict(
        X_test, num_iteration=model.best_iteration
    )

    y_preds.append(y_pred)
    models.append(model)

/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 45814, number of negative: 74186
[LightGBM] [Info] Total Bins 15782
[LightGBM] [Info] Number of data: 120000, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.381783 -> initscore=-0.481986
[LightGBM] [Info] Start training from score -0.481986
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.504949	valid_1's binary_logloss: 0.507524
[20]	training's binary_logloss: 0.437817	valid_1's binary_logloss: 0.44198
[30]	training's binary_logloss

In [33]:

y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.84534

In [34]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)

sub["Survived"] = y_sub
sub.to_csv("./subs/lightgbm_skfold_pseudo_labeled.csv",index=False)